# SW-11-RDFStar

**Navigation** : [<< 10-JSONLD](SW-10-JSONLD.ipynb) | [Index](README.md) | [12-KnowledgeGraphs >>](SW-12-KnowledgeGraphs.ipynb)

## RDF 1.2 (RDF-Star) : Assertions sur des Assertions

### Duree estimee : 45 minutes

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre la motivation de RDF 1.2 (anciennement RDF-Star) et le probleme de la reification classique
2. Utiliser la **reification standard** (`rdf:Statement`) pour annoter des assertions avec rdflib
3. Utiliser les **graphes nommes** (Named Graphs) comme alternative pour le contexte et la provenance
4. Comparer les trois approches : reification classique, graphes nommes, et RDF-Star
5. Appliquer ces patterns a des cas concrets : provenance, confiance, annotations temporelles

### Concepts cles

| Concept | Description |
|---------|-------------|
| RDF 1.2 | Evolution du standard RDF integrant les quoted triples (W3C, 2024) |
| Quoted Triple | Triplet utilise comme sujet ou objet d'un autre triplet : `<< s p o >>` |
| Reification classique | Mecanisme RDF 1.0 pour parler de triplets (4 triplets par assertion) |
| Graphes nommes | Regrouper des triplets dans un contexte identifie par une URI |
| Provenance | Qui a dit quoi ? Source et fiabilite d'une assertion |

### Prerequis
- Python 3.10+
- Notebook [SW-8-PythonRDF](SW-8-PythonRDF.ipynb) (bases rdflib)
- rdflib >= 7.0

### Note sur le support RDF-Star dans rdflib

La syntaxe RDF-Star (`<< s p o >>`, quoted triples) est presentee **conceptuellement** dans ce notebook.
Cependant, rdflib 7.x ne supporte pas encore le parsing Turtle-Star ni la classe `rdflib.term.Triple`.
Nous utilisons donc la **reification standard** et les **graphes nommes** comme implementations pratiques,
tout en montrant comment RDF-Star simplifierait l'ecriture.

---

## Installation des dependances

In [1]:
%pip install -q rdflib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Verifions la version de rdflib installee.

In [2]:
import rdflib

version = rdflib.__version__
major = int(version.split(".")[0])

print(f"rdflib version : {version}")

# Verification du support RDF-Star (Turtle-Star / rdflib.term.Triple)
rdfstar_support = False
try:
    from rdflib.term import Triple as QuotedTriple
    rdfstar_support = True
except ImportError:
    pass

if rdfstar_support:
    print("Support natif RDF-Star (Turtle-Star, QuotedTriple) : disponible.")
else:
    print("Support natif RDF-Star (Turtle-Star, QuotedTriple) : NON disponible.")
    print("Ce notebook utilise la reification standard et les graphes nommes")
    print("comme alternatives fonctionnelles pour enseigner les memes concepts.")

rdflib version : 7.6.0
Support natif RDF-Star (Turtle-Star, QuotedTriple) : NON disponible.
Ce notebook utilise la reification standard et les graphes nommes
comme alternatives fonctionnelles pour enseigner les memes concepts.


---

## 1. Le probleme : pourquoi parler de triplets ?

RDF permet de representer des faits sous forme de triplets. Mais que faire quand on veut **parler d'un triplet lui-meme** ?

Exemples de besoins courants :

| Besoin | Exemple | Ce qu'on veut exprimer |
|--------|---------|------------------------|
| **Provenance** | Source d'une information | "Wikipedia dit que Paris est la capitale de la France" |
| **Confiance** | Degre de certitude | "L'assertion 'X connait Y' a un score de confiance de 0.85" |
| **Temporalite** | Validite dans le temps | "Bob travaille chez Acme depuis 2020" |
| **Attribution** | Qui affirme quoi | "Alice affirme que Bob connait Carol" |
| **Annotation** | Metadata sur un fait | "Ce lien a ete decouvert par le crawler v2" |

En RDF classique (1.0/1.1), le seul mecanisme disponible est la **reification**, qui s'avere lourde et peu pratique.

### 1.1 La reification classique : 4 triplets pour 1 assertion

Pour dire "Alice dit que Bob connait Carol" en RDF classique, il faut creer une ressource intermediaire (un `rdf:Statement`) et decomposer l'assertion en 4 triplets :

```turtle
# Le triplet original qu'on veut annoter :
#   ex:Bob foaf:knows ex:Carol .

# Reification classique (4 triplets supplementaires) :
ex:statement1 rdf:type rdf:Statement .
ex:statement1 rdf:subject ex:Bob .
ex:statement1 rdf:predicate foaf:knows .
ex:statement1 rdf:object ex:Carol .

# Et enfin l'annotation :
ex:statement1 ex:assertedBy ex:Alice .
```

Cela represente **5 triplets supplementaires** pour annoter un seul fait. De plus, la reification ne **garantit pas** que le triplet original existe reellement dans le graphe.

Voyons cela en pratique avec rdflib : la reification classique est verbeuse mais fonctionnelle.

In [3]:
from rdflib import Graph, URIRef, Literal, Namespace, BNode
from rdflib.namespace import RDF, RDFS, FOAF, XSD

EX = Namespace("http://example.org/")

g_reif = Graph()
g_reif.bind("ex", EX)
g_reif.bind("foaf", FOAF)

# Le fait original
g_reif.add((EX.Bob, FOAF.knows, EX.Carol))

# Reification classique : decomposer le triplet
stmt = EX.statement1
g_reif.add((stmt, RDF.type, RDF.Statement))
g_reif.add((stmt, RDF.subject, EX.Bob))
g_reif.add((stmt, RDF.predicate, FOAF.knows))
g_reif.add((stmt, RDF.object, EX.Carol))

# Annotations : qui affirme ce fait, avec quel degre de confiance ?
g_reif.add((stmt, EX.assertedBy, EX.Alice))
g_reif.add((stmt, EX.confidence, Literal(0.85, datatype=XSD.double)))

print(f"Nombre total de triplets : {len(g_reif)}")
print(f"  - Fait original : 1 triplet")
print(f"  - Reification : 4 triplets")
print(f"  - Annotations : 2 triplets")
print(f"  - Total : 7 triplets pour annoter 1 fait")
print()
print(g_reif.serialize(format="turtle"))

Nombre total de triplets : 7
  - Fait original : 1 triplet
  - Reification : 4 triplets
  - Annotations : 2 triplets
  - Total : 7 triplets pour annoter 1 fait

@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:statement1 a rdf:Statement ;
    ex:assertedBy ex:Alice ;
    ex:confidence 8.5e-01 ;
    rdf:object ex:Carol ;
    rdf:predicate foaf:knows ;
    rdf:subject ex:Bob .

ex:Bob foaf:knows ex:Carol .




### Interpretation : cout de la reification

| Approche | Triplets necessaires | Lisibilite | Garantie d'existence |
|----------|---------------------|------------|---------------------|
| Fait seul | 1 | Excellente | Oui |
| Reification classique | 1 + 4 + N annotations | Faible | Non (le Statement est independant) |
| RDF-Star (quoted triple) | 1 + N annotations | Bonne | Configurable |

**Problemes de la reification classique** :
1. **Verbeux** : 4 triplets par assertion reifiee, explosion combinatoire
2. **Deconnecte** : le `rdf:Statement` ne garantit pas que le triplet original existe
3. **Requetes complexes** : les requetes SPARQL pour retrouver les annotations sont longues
4. **Pas standard dans la pratique** : peu de triple stores optimisent la reification

> **Point cle** : RDF-Star (devenu RDF 1.2) resout ces problemes en permettant d'utiliser un triplet directement comme sujet ou objet d'un autre triplet. En attendant un support complet dans rdflib, nous pouvons utiliser la reification de maniere structuree grace a une fonction utilitaire.

---

## 2. RDF 1.2 et les Quoted Triples

### Historique

| Date | Evenement |
|------|----------|
| 2014 | Olaf Hartig propose RDF* (RDF-Star) dans un article academique |
| 2017-2019 | Adoption par des triple stores (Blazegraph, Stardog, Jena) |
| 2021 | Le W3C cree le RDF-Star Community Group |
| 2023 | Le W3C integre RDF-Star dans la specification RDF 1.2 |
| 2024 | RDF 1.2 devient une W3C Recommendation (Candidate) |

### Syntaxe des Quoted Triples

Un **quoted triple** (triplet cite) est un triplet RDF utilise comme sujet ou objet d'un autre triplet. En Turtle-Star, la syntaxe utilise des chevrons doubles `<< ... >>` :

```turtle
# Syntaxe RDF-Star / Turtle-Star (conceptuelle)
<< ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .
<< ex:Bob foaf:knows ex:Carol >> ex:confidence 0.85 .
```

Cela remplacerait les 7 triplets de la reification classique par seulement **2 triplets** (plus le fait original si on veut l'asserter aussi).

### Comparaison visuelle

| Aspect | Reification classique | RDF-Star / RDF 1.2 |
|--------|----------------------|--------------------|
| **Syntaxe** | `ex:stmt rdf:subject ex:Bob .` (4 triplets) | `<< ex:Bob foaf:knows ex:Carol >>` (inline) |
| **Triplets par annotation** | 4 + N | N |
| **Requetes SPARQL** | JOIN sur Statement | Pattern `<< ?s ?p ?o >>` |
| **Support triple stores** | Universel | Croissant (Jena, Blazegraph, Stardog, Oxigraph) |
| **Support rdflib** | Natif (toutes versions) | Pas encore disponible (7.6.0) |

> **Note pratique** : rdflib 7.6.0 ne supporte pas encore la syntaxe Turtle-Star (`<< ... >>`). Dans les sections suivantes, nous implementons les memes concepts via la reification standard, en montrant a chaque fois comment RDF-Star simplifierait l'ecriture.

### 2.1 Fonction utilitaire de reification

Pour faciliter la creation de reifications et reduire la verbosite du code, definissons une fonction `reify()` qui encapsule les 4 triplets necessaires.

En RDF-Star, l'equivalent serait simplement :
```turtle
<< ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .
```

In [4]:
from rdflib import Graph, URIRef, Literal, Namespace, BNode
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")


def reify(graph, subject, predicate, obj, stmt_uri=None):
    """Cree une reification d'un triplet et retourne le noeud Statement.

    En RDF-Star, cela serait simplement << subject predicate obj >>.
    Ici, on cree les 4 triplets rdf:Statement equivalents.

    Args:
        graph: le graphe RDF cible
        subject: sujet du triplet a reifier
        predicate: predicat du triplet a reifier
        obj: objet du triplet a reifier
        stmt_uri: URI optionnelle pour le Statement (BNode par defaut)

    Returns:
        Le noeud (URI ou BNode) representant le rdf:Statement
    """
    stmt = stmt_uri if stmt_uri else BNode()
    graph.add((stmt, RDF.type, RDF.Statement))
    graph.add((stmt, RDF.subject, subject))
    graph.add((stmt, RDF.predicate, predicate))
    graph.add((stmt, RDF.object, obj))
    return stmt


# Demonstration : annoter "Bob connait Carol"
g_demo = Graph()
g_demo.bind("ex", EX)
g_demo.bind("foaf", FOAF)

# Le fait original
g_demo.add((EX.Bob, FOAF.knows, EX.Carol))

# Reification + annotations (equivalent RDF-Star : << ex:Bob foaf:knows ex:Carol >>)
stmt = reify(g_demo, EX.Bob, FOAF.knows, EX.Carol)
g_demo.add((stmt, EX.assertedBy, EX.Alice))
g_demo.add((stmt, EX.confidence, Literal(0.85, datatype=XSD.double)))
g_demo.add((stmt, EX.since, Literal("2020-01-15", datatype=XSD.date)))

print(f"Triplets charges : {len(g_demo)}")
print(f"  - Fait original : 1")
print(f"  - Reification (rdf:Statement) : 4")
print(f"  - Annotations : 3")
print(f"  - Total : {len(g_demo)}")
print()
print("=== Serialisation Turtle ===")
print(g_demo.serialize(format="turtle"))

Triplets charges : 8
  - Fait original : 1
  - Reification (rdf:Statement) : 4
  - Annotations : 3
  - Total : 8

=== Serialisation Turtle ===
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Bob foaf:knows ex:Carol .

[] a rdf:Statement ;
    ex:assertedBy ex:Alice ;
    ex:confidence 8.5e-01 ;
    ex:since "2020-01-15"^^xsd:date ;
    rdf:object ex:Carol ;
    rdf:predicate foaf:knows ;
    rdf:subject ex:Bob .




### Interpretation

La fonction `reify()` simplifie l'ecriture, mais le cout en triplets reste le meme. Comparons :

| Approche | Triplets pour "Bob connait Carol" | Triplets pour 3 annotations | Total |
|----------|----------------------------------|----------------------------|-------|
| Reification classique | 1 + 4 (Statement) | 3 | **8** |
| RDF-Star (si supporte) | 1 | 3 | **4** |

RDF-Star simplifierait cette ecriture avec :
```turtle
ex:Bob foaf:knows ex:Carol .
<< ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .
<< ex:Bob foaf:knows ex:Carol >> ex:confidence "0.85"^^xsd:double .
<< ex:Bob foaf:knows ex:Carol >> ex:since "2020-01-15"^^xsd:date .
```

> **Note technique** : Un quoted triple n'est pas necessairement asserte dans le graphe. `<< ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .` ne signifie pas automatiquement que Bob connait Carol -- seulement qu'Alice le dit. Pour asserter le fait, il faut aussi ajouter `ex:Bob foaf:knows ex:Carol .` explicitement.

---

## 3. Construction programmatique avec reification

Voyons comment construire des annotations sur des triplets de maniere programmatique avec la fonction `reify()`.

### 3.1 Construction programmatique

Grace a la fonction `reify()`, la construction suit un pattern simple :
1. Ajouter le fait original au graphe
2. Appeler `reify()` pour creer le Statement
3. Ajouter les annotations sur le Statement retourne

En RDF-Star, les etapes 2-3 se combineraient : le quoted triple `<< s p o >>` joue directement le role du Statement.

In [5]:
from rdflib import Graph, URIRef, Literal, Namespace, BNode
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

g_prog = Graph()
g_prog.bind("ex", EX)
g_prog.bind("foaf", FOAF)

# Asserter le fait original
g_prog.add((EX.Bob, FOAF.knows, EX.Carol))

# Reifier et annoter
# En RDF-Star, cela serait : << ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .
stmt = reify(g_prog, EX.Bob, FOAF.knows, EX.Carol)
g_prog.add((stmt, EX.assertedBy, EX.Alice))
g_prog.add((stmt, EX.confidence, Literal(0.85, datatype=XSD.double)))
g_prog.add((stmt, EX.source, Literal("Interview 2024-03-15")))

print(f"Graphe construit : {len(g_prog)} triplets")
print(f"  Type du Statement : {type(stmt).__name__}")
print()
print("=== Serialisation Turtle ===")
print(g_prog.serialize(format="turtle"))

Graphe construit : 8 triplets
  Type du Statement : BNode

=== Serialisation Turtle ===
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Bob foaf:knows ex:Carol .

[] a rdf:Statement ;
    ex:assertedBy ex:Alice ;
    ex:confidence 8.5e-01 ;
    ex:source "Interview 2024-03-15" ;
    rdf:object ex:Carol ;
    rdf:predicate foaf:knows ;
    rdf:subject ex:Bob .




### Interpretation

La construction programmatique utilise `reify()` pour encapsuler les 4 triplets de reification :

| Operation | Reification (actuel) | RDF-Star (futur) |
|-----------|---------------------|------------------|
| Creer une reference au triplet | `reify(g, s, p, o)` -> BNode | `QuotedTriple(s, p, o)` |
| Annoter | `g.add((stmt, pred, obj))` | `g.add((qt, pred, obj))` |
| Serialisation | Turtle standard | Turtle-Star (`<< ... >>`) |

> **Avantage de la fonction `reify()`** : elle centralise la creation des 4 triplets de reification, rendant le code plus lisible et moins sujet aux erreurs. Quand rdflib supportera RDF-Star nativement, il suffira de remplacer l'appel a `reify()` par la creation d'un `QuotedTriple`.

### 3.2 Annotations imbriquees (multi-niveaux)

RDF-Star permet l'imbrication : un quoted triple peut lui-meme contenir un quoted triple. Par exemple : "Alice rapporte que Bob croit que Carol connait Dave".

En RDF-Star, cela s'ecrirait :
```turtle
<< << ex:Carol foaf:knows ex:Dave >> ex:believedBy ex:Bob >> ex:reportedBy ex:Alice .
```

Avec la reification, on chaine les Statements. Construisons cet exemple.

In [6]:
from rdflib import Graph, Namespace, Literal, BNode
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

g_nested = Graph()
g_nested.bind("ex", EX)
g_nested.bind("foaf", FOAF)

# Niveau 1 : Carol connait Dave (fait de base)
g_nested.add((EX.Carol, FOAF.knows, EX.Dave))

# Niveau 2 : Bob croit ce fait
# En RDF-Star : << ex:Carol foaf:knows ex:Dave >> ex:believedBy ex:Bob .
stmt_level1 = reify(g_nested, EX.Carol, FOAF.knows, EX.Dave)
g_nested.add((stmt_level1, EX.believedBy, EX.Bob))

# Niveau 3 : Alice rapporte la croyance de Bob
# En RDF-Star : << << ... >> ex:believedBy ex:Bob >> ex:reportedBy ex:Alice .
# Avec la reification, on reifie le Statement de niveau 2
stmt_level2 = reify(g_nested, stmt_level1, EX.believedBy, EX.Bob)
g_nested.add((stmt_level2, EX.reportedBy, EX.Alice))
g_nested.add((stmt_level2, EX.reportDate, Literal("2024-06-01", datatype=XSD.date)))

print(f"Triplets charges : {len(g_nested)}")
print(f"  - Fait de base : 1")
print(f"  - Reification niveau 1 : 4 + 1 annotation")
print(f"  - Reification niveau 2 : 4 + 2 annotations")
print(f"  - Total : {len(g_nested)}")
print()
print("=== Serialisation Turtle ===")
print(g_nested.serialize(format="turtle"))

Triplets charges : 12
  - Fait de base : 1
  - Reification niveau 1 : 4 + 1 annotation
  - Reification niveau 2 : 4 + 2 annotations
  - Total : 12

=== Serialisation Turtle ===
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Carol foaf:knows ex:Dave .

[] a rdf:Statement ;
    ex:reportDate "2024-06-01"^^xsd:date ;
    ex:reportedBy ex:Alice ;
    rdf:object ex:Bob ;
    rdf:predicate ex:believedBy ;
    rdf:subject [ a rdf:Statement ;
            ex:believedBy ex:Bob ;
            rdf:object ex:Dave ;
            rdf:predicate foaf:knows ;
            rdf:subject ex:Carol ] .




### Interpretation : imbrication

Chaque niveau d'imbrication ajoute du contexte sans modifier le fait de base :

| Niveau | Reification | RDF-Star equivalent | Signification |
|--------|-------------|--------------------|--------------|
| 1 | `Carol foaf:knows Dave` | `Carol foaf:knows Dave` | Fait asserte |
| 2 | `stmt1 believedBy Bob` | `<< Carol knows Dave >> believedBy Bob` | Bob croit ce fait |
| 3 | `stmt2 reportedBy Alice` | `<< << ... >> believedBy Bob >> reportedBy Alice` | Alice rapporte la croyance |

Le cout en triplets est eleve avec la reification (12 triplets) contre seulement 4 avec RDF-Star.

> **Attention** : L'imbrication profonde (>2 niveaux) rend les requetes SPARQL complexes, que ce soit en reification ou en RDF-Star. En pratique, un seul niveau d'imbrication couvre la majorite des cas d'usage.

---

## 4. Interroger les annotations avec SPARQL

Le pendant de SPARQL-Star pour la reification classique utilise des patterns de jointure sur les proprietes `rdf:subject`, `rdf:predicate` et `rdf:object`.

### Comparaison des syntaxes

| Pattern | SPARQL-Star (conceptuel) | SPARQL avec reification |
|---------|------------------------|------------------------|
| Trouver les annotations | `<< ?s ?p ?o >> ?annPred ?annObj .` | `?stmt rdf:subject ?s ; rdf:predicate ?p ; rdf:object ?o ; ?annPred ?annObj .` |
| Triplet specifique | `<< ex:Bob foaf:knows ?who >> ?p ?o .` | `?stmt rdf:subject ex:Bob ; rdf:predicate foaf:knows ; rdf:object ?who .` |
| Annotation specifique | `<< ?s ?p ?o >> ex:confidence ?c .` | `?stmt rdf:subject ?s ; rdf:predicate ?p ; rdf:object ?o ; ex:confidence ?c .` |

### 4.1 Construire un graphe de connaissances annote

Creons un graphe plus riche avec des relations annotees : provenance, confiance et dates.

In [7]:
from rdflib import Graph, Namespace, Literal, URIRef, BNode
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

g_kg = Graph()
g_kg.bind("ex", EX)
g_kg.bind("foaf", FOAF)

# === Personnes ===
for name, uri in [("Alice Dupont", EX.Alice), ("Bob Martin", EX.Bob),
                   ("Carol Laurent", EX.Carol), ("Dave Petit", EX.Dave)]:
    g_kg.add((uri, RDF.type, FOAF.Person))
    g_kg.add((uri, FOAF.name, Literal(name)))

# === Fait 1 : Alice connait Bob (haute confiance, source LinkedIn) ===
# RDF-Star : << ex:Alice foaf:knows ex:Bob >> ex:confidence "0.95"^^xsd:double .
g_kg.add((EX.Alice, FOAF.knows, EX.Bob))
stmt1 = reify(g_kg, EX.Alice, FOAF.knows, EX.Bob)
g_kg.add((stmt1, EX.confidence, Literal(0.95, datatype=XSD.double)))
g_kg.add((stmt1, EX.source, Literal("LinkedIn")))
g_kg.add((stmt1, EX.since, Literal("2019-03-01", datatype=XSD.date)))

# === Fait 2 : Bob connait Carol (confiance moyenne, source Twitter) ===
g_kg.add((EX.Bob, FOAF.knows, EX.Carol))
stmt2 = reify(g_kg, EX.Bob, FOAF.knows, EX.Carol)
g_kg.add((stmt2, EX.confidence, Literal(0.60, datatype=XSD.double)))
g_kg.add((stmt2, EX.source, Literal("Twitter")))
g_kg.add((stmt2, EX.since, Literal("2021-07-15", datatype=XSD.date)))

# === Fait 3 : Carol connait Dave (basse confiance, source rumeur) ===
g_kg.add((EX.Carol, FOAF.knows, EX.Dave))
stmt3 = reify(g_kg, EX.Carol, FOAF.knows, EX.Dave)
g_kg.add((stmt3, EX.confidence, Literal(0.30, datatype=XSD.double)))
g_kg.add((stmt3, EX.source, Literal("Rumeur")))

# === Fait 4 : Alice connait Dave (NON asserte, seulement affirme par Bob) ===
# On ne fait PAS g_kg.add((EX.Alice, FOAF.knows, EX.Dave))
stmt4 = reify(g_kg, EX.Alice, FOAF.knows, EX.Dave)
g_kg.add((stmt4, EX.assertedBy, EX.Bob))
g_kg.add((stmt4, EX.confidence, Literal(0.50, datatype=XSD.double)))

print(f"Graphe de connaissances : {len(g_kg)} triplets")
print()
print(g_kg.serialize(format="turtle"))

Graphe de connaissances : 37 triplets

@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Alice a foaf:Person ;
    foaf:knows ex:Bob ;
    foaf:name "Alice Dupont" .

ex:Carol a foaf:Person ;
    foaf:knows ex:Dave ;
    foaf:name "Carol Laurent" .

ex:Dave a foaf:Person ;
    foaf:name "Dave Petit" .

ex:Bob a foaf:Person ;
    foaf:knows ex:Carol ;
    foaf:name "Bob Martin" .

[] a rdf:Statement ;
    ex:confidence 6e-01 ;
    ex:since "2021-07-15"^^xsd:date ;
    ex:source "Twitter" ;
    rdf:object ex:Carol ;
    rdf:predicate foaf:knows ;
    rdf:subject ex:Bob .

[] a rdf:Statement ;
    ex:confidence 9.5e-01 ;
    ex:since "2019-03-01"^^xsd:date ;
    ex:source "LinkedIn" ;
    rdf:object ex:Bob ;
    rdf:predicate foaf:knows ;
    rdf:subject ex:Alice .

[] a rdf:Statement ;
    ex:confidence 3e-01 ;
    ex:source "Rumeur" ;
    rdf

### Interpretation : graphe de connaissances annote

Ce graphe illustre un reseau social avec des annotations de qualite :

| Relation | Confiance | Source | Depuis | Asserte ? |
|----------|-----------|--------|--------|----------|
| Alice connait Bob | 0.95 | LinkedIn | 2019-03-01 | Oui |
| Bob connait Carol | 0.60 | Twitter | 2021-07-15 | Oui |
| Carol connait Dave | 0.30 | Rumeur | - | Oui |
| Alice connait Dave | 0.50 | - | - | **Non** (seulement affirme par Bob) |

> **Point cle** : Le fait 4 n'est **pas asserte** dans le graphe (`ex:Alice foaf:knows ex:Dave .` n'apparait pas). Seul le `rdf:Statement` et ses annotations existent. Cela signifie que le graphe ne dit pas qu'Alice connait Dave -- il dit seulement que Bob le pretend.

En RDF-Star, la meme distinction existe : `<< ex:Alice foaf:knows ex:Dave >> ex:assertedBy ex:Bob .` n'asserte pas le fait non plus.

### 4.2 Requetes SPARQL sur les reifications

Interrogeons le graphe avec SPARQL standard pour extraire les annotations via les patterns de reification.

In [8]:
# Requete 1 : Toutes les relations annotees avec confiance
# En SPARQL-Star, on ecrirait : << ?person1 foaf:knows ?person2 >> ex:confidence ?confidence .
# Avec la reification, on joint sur rdf:subject, rdf:predicate, rdf:object

query_all = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?person1 ?person2 ?confidence ?source ?since
WHERE {
    ?stmt a rdf:Statement ;
          rdf:subject ?person1 ;
          rdf:predicate foaf:knows ;
          rdf:object ?person2 ;
          ex:confidence ?confidence .
    OPTIONAL { ?stmt ex:source ?source . }
    OPTIONAL { ?stmt ex:since ?since . }
}
ORDER BY DESC(?confidence)
"""

results = g_kg.query(query_all)

print("=== Relations annotees (triees par confiance) ===")
print(f"{'Personne 1':<15} {'Personne 2':<15} {'Confiance':>10} {'Source':<12} {'Depuis'}")
print("-" * 70)
for row in results:
    p1 = str(row.person1).split("/")[-1]
    p2 = str(row.person2).split("/")[-1]
    conf = f"{float(row.confidence):.2f}"
    src = str(row.source) if row.source else "-"
    since = str(row.since) if row.since else "-"
    print(f"{p1:<15} {p2:<15} {conf:>10} {src:<12} {since}")

=== Relations annotees (triees par confiance) ===
Personne 1      Personne 2       Confiance Source       Depuis
----------------------------------------------------------------------
Alice           Bob                   0.95 LinkedIn     2019-03-01
Bob             Carol                 0.60 Twitter      2021-07-15
Alice           Dave                  0.50 -            -
Carol           Dave                  0.30 Rumeur       -


### 4.3 Filtrer par score de confiance

Un cas d'usage important : ne retenir que les relations ayant un score de confiance superieur a un seuil. Cela permet de construire une vue "fiable" du graphe.

In [9]:
# Requete : Relations a haute confiance (>= 0.60)
# En SPARQL-Star : << ?p1 foaf:knows ?p2 >> ex:confidence ?c . FILTER(?c >= 0.60)

query_high = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?person1 ?person2 ?confidence ?source
WHERE {
    ?stmt a rdf:Statement ;
          rdf:subject ?person1 ;
          rdf:predicate foaf:knows ;
          rdf:object ?person2 ;
          ex:confidence ?confidence .
    FILTER (?confidence >= "0.60"^^xsd:double)
    OPTIONAL { ?stmt ex:source ?source . }
}
ORDER BY DESC(?confidence)
"""

# Requete : Relations a basse confiance (< 0.60)
query_low = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?person1 ?person2 ?confidence ?source
WHERE {
    ?stmt a rdf:Statement ;
          rdf:subject ?person1 ;
          rdf:predicate foaf:knows ;
          rdf:object ?person2 ;
          ex:confidence ?confidence .
    FILTER (?confidence < "0.60"^^xsd:double)
    OPTIONAL { ?stmt ex:source ?source . }
}
ORDER BY ?confidence
"""

print("=== Relations FIABLES (confiance >= 0.60) ===")
for row in g_kg.query(query_high):
    p1 = str(row.person1).split("/")[-1]
    p2 = str(row.person2).split("/")[-1]
    src = str(row.source) if row.source else "-"
    print(f"  {p1} connait {p2} (confiance={float(row.confidence):.2f}, source={src})")

print()
print("=== Relations INCERTAINES (confiance < 0.60) ===")
for row in g_kg.query(query_low):
    p1 = str(row.person1).split("/")[-1]
    p2 = str(row.person2).split("/")[-1]
    src = str(row.source) if row.source else "-"
    print(f"  {p1} connait {p2} (confiance={float(row.confidence):.2f}, source={src})")

=== Relations FIABLES (confiance >= 0.60) ===


  Alice connait Bob (confiance=0.95, source=LinkedIn)
  Bob connait Carol (confiance=0.60, source=Twitter)

=== Relations INCERTAINES (confiance < 0.60) ===


  Carol connait Dave (confiance=0.30, source=Rumeur)
  Alice connait Dave (confiance=0.50, source=-)


### Interpretation : filtrage par confiance

Le filtrage separe les faits fiables des faits incertains :

| Categorie | Relations | Usage recommande |
|-----------|-----------|------------------|
| Haute confiance (>= 0.60) | Alice-Bob, Bob-Carol | Utiliser pour le raisonnement |
| Basse confiance (< 0.60) | Carol-Dave, Alice-Dave | A verifier, ne pas propager |

Ce type de filtrage est essentiel dans les **graphes de connaissances** : on peut construire une vue "fiable" du graphe en excluant les assertions douteuses.

> **Application concrete** : Dans un systeme de recommandation, on ne recommanderait un contact qu'a partir de relations de confiance >= 0.70.

---

## 5. Cas d'usage concrets

Explorons trois cas d'usage representatifs pour l'annotation de triplets.

### 5.1 Provenance et attribution

Le cas d'usage le plus naturel : enregistrer **qui** a produit une information, **quand**, et avec **quelle methode**. Ce pattern s'integre naturellement avec le vocabulaire PROV-O du W3C.

En RDF-Star, cela s'ecrirait de maniere compacte :
```turtle
<< ex:Paris schema:population "2161000"^^xsd:integer >>
    prov:wasDerivedFrom ex:INSEE_2023 ;
    prov:generatedAtTime "2023-12-01"^^xsd:date .
```

In [10]:
from rdflib import Graph, Namespace, Literal, BNode
from rdflib.namespace import RDF, RDFS, XSD

EX = Namespace("http://example.org/")
PROV = Namespace("http://www.w3.org/ns/prov#")
SDO = Namespace("https://schema.org/")

g_prov = Graph()
g_prov.bind("ex", EX)
g_prov.bind("prov", PROV)
g_prov.bind("schema", SDO)

# Fait 1 : Paris a une population de 2,1 millions
pop_value = Literal(2161000, datatype=XSD.integer)
g_prov.add((EX.Paris, SDO.population, pop_value))

# Provenance du fait (reification)
stmt_pop = reify(g_prov, EX.Paris, SDO.population, pop_value)
g_prov.add((stmt_pop, PROV.wasDerivedFrom, EX.INSEE_2023))
g_prov.add((stmt_pop, PROV.generatedAtTime, Literal("2023-12-01", datatype=XSD.date)))
g_prov.add((stmt_pop, EX.method, Literal("Recensement")))
g_prov.add((stmt_pop, EX.accuracy, Literal("haute")))

# Fait 2 : Paris a une superficie de 105.4 km2
area_value = Literal(105.4, datatype=XSD.double)
g_prov.add((EX.Paris, SDO.area, area_value))

stmt_area = reify(g_prov, EX.Paris, SDO.area, area_value)
g_prov.add((stmt_area, PROV.wasDerivedFrom, EX.IGN_2023))
g_prov.add((stmt_area, PROV.generatedAtTime, Literal("2023-06-15", datatype=XSD.date)))
g_prov.add((stmt_area, EX.method, Literal("Mesure cadastrale")))

print(f"Graphe de provenance : {len(g_prov)} triplets")
print()

# Requete : quels faits, avec quelle source ?
query_prov = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>

SELECT ?subject ?property ?value ?source ?date ?method
WHERE {
    ?stmt a rdf:Statement ;
          rdf:subject ?subject ;
          rdf:predicate ?property ;
          rdf:object ?value ;
          prov:wasDerivedFrom ?source .
    OPTIONAL { ?stmt prov:generatedAtTime ?date . }
    OPTIONAL { ?stmt ex:method ?method . }
}
"""

print("=== Faits avec provenance ===")
print(f"{'Sujet':<10} {'Propriete':<20} {'Valeur':<12} {'Source':<15} {'Date':<12} {'Methode'}")
print("-" * 85)
for row in g_prov.query(query_prov):
    subj = str(row.subject).split("/")[-1]
    prop = str(row.property).split("/")[-1]
    src = str(row.source).split("/")[-1]
    date = str(row.date) if row.date else "-"
    method = str(row.method) if row.method else "-"
    print(f"{subj:<10} {prop:<20} {str(row.value):<12} {src:<15} {date:<12} {method}")

Graphe de provenance : 17 triplets

=== Faits avec provenance ===
Sujet      Propriete            Valeur       Source          Date         Methode
-------------------------------------------------------------------------------------


Paris      population           2161000      INSEE_2023      2023-12-01   Recensement
Paris      area                 105.4        IGN_2023        2023-06-15   Mesure cadastrale


### Interpretation : provenance

| Fait | Source | Date | Methode |
|------|--------|------|--------|
| Population de Paris = 2,161,000 | INSEE 2023 | 2023-12-01 | Recensement |
| Superficie de Paris = 105.4 km2 | IGN 2023 | 2023-06-15 | Mesure cadastrale |

RDF-Star simplifierait cette ecriture en eliminant les 4 triplets de reification par fait :
```turtle
<< ex:Paris schema:population "2161000"^^xsd:integer >>
    prov:wasDerivedFrom ex:INSEE_2023 ;
    prov:generatedAtTime "2023-12-01"^^xsd:date ;
    ex:method "Recensement" ;
    ex:accuracy "haute" .
```

En reification classique, il faut 2 x 4 = 8 triplets supplementaires pour les Statements seuls.

### 5.2 Annotations temporelles

La reification est utile pour exprimer la validite temporelle des faits : un emploi, un statut, une relation qui change au fil du temps.

En RDF-Star, cela s'ecrirait :
```turtle
<< ex:Marie schema:worksFor ex:Sorbonne >> ex:startDate "2020-09-01"^^xsd:date .
<< ex:Marie schema:worksFor ex:Sorbonne >> ex:role "Professeure" .
```

In [11]:
from rdflib import Graph, Namespace, Literal, BNode
from rdflib.namespace import RDF, XSD

EX = Namespace("http://example.org/")
SDO = Namespace("https://schema.org/")

g_temporal = Graph()
g_temporal.bind("ex", EX)
g_temporal.bind("schema", SDO)

# Personnes et organisations
for name, uri, rdf_type in [
    ("Marie", EX.Marie, SDO.Person),
    ("Sorbonne", EX.Sorbonne, SDO.Organization),
    ("CNRS", EX.CNRS, SDO.Organization),
    ("MIT", EX.MIT, SDO.Organization),
]:
    g_temporal.add((uri, RDF.type, rdf_type))
    g_temporal.add((uri, SDO.name, Literal(name)))

# Emploi actuel : Marie -> Sorbonne (asserte)
g_temporal.add((EX.Marie, SDO.worksFor, EX.Sorbonne))
stmt_sorb = reify(g_temporal, EX.Marie, SDO.worksFor, EX.Sorbonne)
g_temporal.add((stmt_sorb, EX.startDate, Literal("2020-09-01", datatype=XSD.date)))
g_temporal.add((stmt_sorb, EX.role, Literal("Professeure")))
g_temporal.add((stmt_sorb, EX.status, Literal("actif")))

# Emploi precedent 1 : Marie -> CNRS (non asserte, termine)
stmt_cnrs = reify(g_temporal, EX.Marie, SDO.worksFor, EX.CNRS)
g_temporal.add((stmt_cnrs, EX.startDate, Literal("2015-01-15", datatype=XSD.date)))
g_temporal.add((stmt_cnrs, EX.endDate, Literal("2020-08-31", datatype=XSD.date)))
g_temporal.add((stmt_cnrs, EX.role, Literal("Chercheuse")))
g_temporal.add((stmt_cnrs, EX.status, Literal("termine")))

# Emploi precedent 2 : Marie -> MIT (non asserte, termine)
stmt_mit = reify(g_temporal, EX.Marie, SDO.worksFor, EX.MIT)
g_temporal.add((stmt_mit, EX.startDate, Literal("2012-09-01", datatype=XSD.date)))
g_temporal.add((stmt_mit, EX.endDate, Literal("2014-12-31", datatype=XSD.date)))
g_temporal.add((stmt_mit, EX.role, Literal("Post-doc")))
g_temporal.add((stmt_mit, EX.status, Literal("termine")))

print(f"Graphe temporel : {len(g_temporal)} triplets")
print()

# Requete : parcours professionnel chronologique
query_career = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>

SELECT ?orgName ?role ?startDate ?endDate ?status
WHERE {
    ?stmt a rdf:Statement ;
          rdf:subject ex:Marie ;
          rdf:predicate schema:worksFor ;
          rdf:object ?org ;
          ex:role ?role ;
          ex:startDate ?startDate ;
          ex:status ?status .
    OPTIONAL { ?stmt ex:endDate ?endDate . }
    ?org schema:name ?orgName .
}
ORDER BY ?startDate
"""

print("=== Parcours professionnel de Marie ===")
print(f"{'Organisation':<15} {'Role':<15} {'Debut':<12} {'Fin':<12} {'Statut'}")
print("-" * 65)
for row in g_temporal.query(query_career):
    end = str(row.endDate) if row.endDate else "en cours"
    print(f"{str(row.orgName):<15} {str(row.role):<15} {str(row.startDate):<12} {end:<12} {str(row.status)}")

Graphe temporel : 32 triplets

=== Parcours professionnel de Marie ===
Organisation    Role            Debut        Fin          Statut
-----------------------------------------------------------------
MIT             Post-doc        2012-09-01   2014-12-31   termine
CNRS            Chercheuse      2015-01-15   2020-08-31   termine
Sorbonne        Professeure     2020-09-01   en cours     actif


### Interpretation : annotations temporelles

Le parcours professionnel est represente de maniere structuree :

| Periode | Organisation | Role | Statut |
|---------|-------------|------|--------|
| 2012-2014 | MIT | Post-doc | Termine |
| 2015-2020 | CNRS | Chercheuse | Termine |
| 2020-... | Sorbonne | Professeure | Actif |

**Sans RDF-Star**, cet historique necessite soit :
- La **reification classique** (ce que nous faisons ici) : 4 triplets par relation + annotations
- Des **blank nodes intermediaires** (ex: `ex:emploi1 ex:employer ex:MIT ; ex:role "Post-doc" .`) ce qui perd le lien direct `Marie -> worksFor -> MIT`

RDF-Star simplifierait cette ecriture en annotant directement :
```turtle
<< ex:Marie schema:worksFor ex:CNRS >> ex:startDate "2015-01-15"^^xsd:date ;
                                        ex:endDate "2020-08-31"^^xsd:date ;
                                        ex:role "Chercheuse" ;
                                        ex:status "termine" .
```

### 5.3 Fusion multi-sources avec scores de confiance

Dans un graphe de connaissances construit a partir de sources multiples, chaque source peut affirmer des faits contradictoires. La reification permet de conserver toutes les versions et de les departager.

In [12]:
from rdflib import Graph, Namespace, Literal, BNode
from rdflib.namespace import RDF, XSD

EX = Namespace("http://example.org/")
SDO = Namespace("https://schema.org/")

g_multi = Graph()
g_multi.bind("ex", EX)
g_multi.bind("schema", SDO)

g_multi.add((EX.Lyon, RDF.type, SDO.City))
g_multi.add((EX.Lyon, SDO.name, Literal("Lyon")))

# Source 1 : INSEE (haute confiance)
# RDF-Star : << ex:Lyon schema:population "522250"^^xsd:integer >> ex:source "INSEE" .
pop_insee = Literal(522250, datatype=XSD.integer)
stmt_insee = reify(g_multi, EX.Lyon, SDO.population, pop_insee)
g_multi.add((stmt_insee, EX.source, Literal("INSEE")))
g_multi.add((stmt_insee, EX.confidence, Literal(0.95, datatype=XSD.double)))
g_multi.add((stmt_insee, EX.year, Literal("2023", datatype=XSD.gYear)))

# Source 2 : Wikipedia (confiance moyenne)
pop_wiki = Literal(516092, datatype=XSD.integer)
stmt_wiki = reify(g_multi, EX.Lyon, SDO.population, pop_wiki)
g_multi.add((stmt_wiki, EX.source, Literal("Wikipedia")))
g_multi.add((stmt_wiki, EX.confidence, Literal(0.70, datatype=XSD.double)))
g_multi.add((stmt_wiki, EX.year, Literal("2021", datatype=XSD.gYear)))

# Source 3 : Estimation locale (basse confiance)
pop_mairie = Literal(530000, datatype=XSD.integer)
stmt_mairie = reify(g_multi, EX.Lyon, SDO.population, pop_mairie)
g_multi.add((stmt_mairie, EX.source, Literal("Estimation mairie")))
g_multi.add((stmt_mairie, EX.confidence, Literal(0.40, datatype=XSD.double)))
g_multi.add((stmt_mairie, EX.year, Literal("2024", datatype=XSD.gYear)))

print(f"Graphe multi-sources : {len(g_multi)} triplets")
print()

# Requete : toutes les valeurs avec leurs sources
query_sources = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>

SELECT ?pop ?source ?confidence ?year
WHERE {
    ?stmt a rdf:Statement ;
          rdf:subject ex:Lyon ;
          rdf:predicate schema:population ;
          rdf:object ?pop ;
          ex:source ?source ;
          ex:confidence ?confidence ;
          ex:year ?year .
}
ORDER BY DESC(?confidence)
"""

print("=== Population de Lyon selon differentes sources ===")
print(f"{'Source':<20} {'Population':>12} {'Confiance':>10} {'Annee':>6}")
print("-" * 52)
for row in g_multi.query(query_sources):
    pop = f"{int(row.pop):,d}"
    conf = f"{float(row.confidence):.2f}"
    print(f"{str(row.source):<20} {pop:>12} {conf:>10} {str(row.year):>6}")

# Valeur la plus fiable
query_best = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>

SELECT ?pop ?source ?confidence
WHERE {
    ?stmt a rdf:Statement ;
          rdf:subject ex:Lyon ;
          rdf:predicate schema:population ;
          rdf:object ?pop ;
          ex:confidence ?confidence ;
          ex:source ?source .
}
ORDER BY DESC(?confidence)
LIMIT 1
"""

print()
for row in g_multi.query(query_best):
    print(f"Valeur retenue (confiance max) : {int(row.pop):,d} (source: {row.source})")

Graphe multi-sources : 23 triplets

=== Population de Lyon selon differentes sources ===
Source                 Population  Confiance  Annee
----------------------------------------------------
INSEE                     522,250       0.95   2023
Wikipedia                 516,092       0.70   2021
Estimation mairie         530,000       0.40   2024

Valeur retenue (confiance max) : 522,250 (source: INSEE)


### Interpretation : fusion multi-sources

Ce pattern est fondamental pour les graphes de connaissances industriels :

1. **Conserver toutes les valeurs** : chaque source a sa propre estimation
2. **Annoter la confiance** : permettre un filtrage par fiabilite
3. **Tracer la provenance** : savoir d'ou vient chaque information
4. **Selectionner la meilleure** : prendre la valeur la plus fiable pour un usage donne

| Strategie de selection | Requete | Usage |
|----------------------|---------|-------|
| Plus haute confiance | `ORDER BY DESC(?confidence) LIMIT 1` | Vue par defaut |
| Plus recente | `ORDER BY DESC(?year) LIMIT 1` | Donnees a jour |
| Moyenne ponderee | Calcul hors SPARQL | Estimation consolidee |

---

## 6. Alternative : les Graphes Nommes (Named Graphs)

Les **graphes nommes** offrent une autre approche pour associer du contexte a des triplets.
Au lieu d'annoter un triplet individuel, on place les triplets dans un graphe identifie par une URI,
puis on annote cette URI.

### Comparaison des approches

| Aspect | Reification | Graphes nommes | RDF-Star |
|--------|------------|----------------|----------|
| **Granularite** | Par triplet | Par groupe de triplets | Par triplet |
| **Verbosite** | 4 triplets / annotation | 1 URI par contexte | Inline |
| **Modele de donnees** | Triplets | Quadruplets (quads) | Triplets etendus |
| **Support rdflib** | Oui (toutes versions) | Oui (`Dataset`) | Non (7.6.0) |
| **Support SPARQL** | Standard | `GRAPH ?g { ... }` | SPARQL-Star |
| **Cas d'usage ideal** | Annotation fine | Provenance par lot | Annotation fine |

### 6.1 Graphes nommes avec rdflib Dataset

rdflib fournit la classe `Dataset` pour gerer des graphes nommes. Chaque graphe est identifie par une URI et contient un ensemble de triplets.

In [13]:
from rdflib import Dataset, URIRef, Literal, Namespace
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

ds = Dataset()
ds.bind("ex", EX)
ds.bind("foaf", FOAF)

# Creer un graphe nomme pour le contexte "assertion d'Alice"
ctx_alice = URIRef("http://example.org/context/alice-assertion")
g_alice = ds.graph(ctx_alice)

# Les triplets sont dans le graphe nomme
g_alice.add((EX.Bob, FOAF.knows, EX.Carol))
g_alice.add((EX.Bob, FOAF.knows, EX.Dave))

# Les annotations sont dans le graphe par defaut
ds.default_graph.add((ctx_alice, RDF.type, EX.Assertion))
ds.default_graph.add((ctx_alice, EX.assertedBy, EX.Alice))
ds.default_graph.add((ctx_alice, EX.confidence, Literal(0.85, datatype=XSD.double)))
ds.default_graph.add((ctx_alice, EX.date, Literal("2024-03-15", datatype=XSD.date)))

# Second contexte : assertion de Bob
ctx_bob = URIRef("http://example.org/context/bob-assertion")
g_bob = ds.graph(ctx_bob)
g_bob.add((EX.Carol, FOAF.knows, EX.Eve))

ds.default_graph.add((ctx_bob, RDF.type, EX.Assertion))
ds.default_graph.add((ctx_bob, EX.assertedBy, EX.Bob))
ds.default_graph.add((ctx_bob, EX.confidence, Literal(0.60, datatype=XSD.double)))

print("=== Graphe par defaut (metadata) ===")
for s, p, o in sorted(ds.default_graph, key=lambda t: str(t[0])):
    s_short = str(s).replace("http://example.org/", "ex:")
    p_short = str(p).replace("http://example.org/", "ex:").replace(
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#", "rdf:")
    o_short = str(o).replace("http://example.org/", "ex:")
    print(f"  {s_short} {p_short} {o_short}")

print()
print("=== Graphe nomme : alice-assertion ===")
for s, p, o in g_alice:
    s_short = str(s).replace("http://example.org/", "ex:")
    p_short = str(p).replace("http://xmlns.com/foaf/0.1/", "foaf:")
    o_short = str(o).replace("http://example.org/", "ex:")
    print(f"  {s_short} {p_short} {o_short}")

print()
print("=== Graphe nomme : bob-assertion ===")
for s, p, o in g_bob:
    s_short = str(s).replace("http://example.org/", "ex:")
    p_short = str(p).replace("http://xmlns.com/foaf/0.1/", "foaf:")
    o_short = str(o).replace("http://example.org/", "ex:")
    print(f"  {s_short} {p_short} {o_short}")

=== Graphe par defaut (metadata) ===
  ex:context/alice-assertion rdf:type ex:Assertion
  ex:context/alice-assertion ex:date 2024-03-15
  ex:context/alice-assertion ex:assertedBy ex:Alice
  ex:context/alice-assertion ex:confidence 0.85
  ex:context/bob-assertion ex:assertedBy ex:Bob
  ex:context/bob-assertion ex:confidence 0.6
  ex:context/bob-assertion rdf:type ex:Assertion

=== Graphe nomme : alice-assertion ===
  ex:Bob foaf:knows ex:Dave
  ex:Bob foaf:knows ex:Carol

=== Graphe nomme : bob-assertion ===
  ex:Carol foaf:knows ex:Eve


### Interpretation : graphes nommes

Les graphes nommes regroupent des triplets par contexte :

| Graphe | Contenu | Annotations |
|--------|---------|-------------|
| `ex:context/alice-assertion` | Bob connait Carol, Bob connait Dave | Asserte par Alice, confiance 0.85 |
| `ex:context/bob-assertion` | Carol connait Eve | Asserte par Bob, confiance 0.60 |

**Avantages des graphes nommes** :
- Annotation de **groupes** de triplets (pas individuel)
- Pas de triplets supplementaires par fait annote
- Support natif dans tous les triple stores (SPARQL `GRAPH`)

**Limites** :
- Granularite trop grosse si on veut annoter un seul triplet (il faut un graphe par triplet)
- Un triplet ne peut etre que dans un seul graphe nomme a la fois

> **Quand utiliser quoi ?** : Les graphes nommes conviennent pour la provenance par lot ("tous ces faits viennent de cette source"). La reification (ou RDF-Star) convient pour les annotations fines par triplet.

---

## 7. Serialisation et interoperabilite

### Formats et support des annotations de triplets

| Format | Reification | Graphes nommes | RDF-Star |
|--------|------------|----------------|----------|
| **Turtle** | Oui (standard) | Non | Turtle-Star (extension) |
| **TriG** | Oui | Oui (natif) | TriG-Star (extension) |
| **N-Quads** | Oui | Oui (natif) | Non |
| **JSON-LD** | Oui | Oui (`@graph`) | En discussion |
| **RDF/XML** | Oui | Non | Non |

### Serialisation d'un graphe avec reification

Testons les formats disponibles dans rdflib pour un graphe contenant des reifications.

In [14]:
from rdflib import Graph, Namespace, Literal, BNode
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

g_ser = Graph()
g_ser.bind("ex", EX)
g_ser.bind("foaf", FOAF)

# Fait + reification + annotation
g_ser.add((EX.Alice, FOAF.knows, EX.Bob))
stmt = reify(g_ser, EX.Alice, FOAF.knows, EX.Bob)
g_ser.add((stmt, EX.confidence, Literal(0.9, datatype=XSD.double)))

formats_to_test = [
    ("Turtle", "turtle"),
    ("N-Triples", "nt"),
    ("RDF/XML", "xml"),
]

for label, fmt in formats_to_test:
    try:
        output = g_ser.serialize(format=fmt)
        print(f"=== {label} ===")
        print(output.strip())
        print()
    except Exception as e:
        print(f"=== {label} : Erreur ===")
        print(f"  {e}")
        print()

# JSON-LD
try:
    jsonld_out = g_ser.serialize(format="json-ld", indent=2)
    print("=== JSON-LD ===")
    print(jsonld_out.strip())
except Exception as e:
    print(f"=== JSON-LD : {e} ===")

=== Turtle ===
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Alice foaf:knows ex:Bob .

[] a rdf:Statement ;
    ex:confidence 9e-01 ;
    rdf:object ex:Bob ;
    rdf:predicate foaf:knows ;
    rdf:subject ex:Alice .

=== N-Triples ===
_:N6dd3657e2b0f41a6bcae8350d38e7345 <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://xmlns.com/foaf/0.1/knows> .
_:N6dd3657e2b0f41a6bcae8350d38e7345 <http://example.org/confidence> "0.9"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://example.org/Alice> <http://xmlns.com/foaf/0.1/knows> <http://example.org/Bob> .
_:N6dd3657e2b0f41a6bcae8350d38e7345 <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> <http://example.org/Bob> .
_:N6dd3657e2b0f41a6bcae8350d38e7345 <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <http://example.org/Alice> .
_:N6dd3657e2b0f41a6bcae8350d38e7345 <http://ww

### Interpretation : formats de serialisation

- **Turtle** : la reification est representee avec des blank nodes et les proprietes `rdf:subject`, `rdf:predicate`, `rdf:object`
- **N-Triples** : chaque triplet est explicite, les blank nodes ont des identifiants generes
- **RDF/XML** : la reification est supportee via la syntaxe `rdf:Description`
- **JSON-LD** : la reification est representee comme un objet avec `@type: rdf:Statement`

Quand le support Turtle-Star sera disponible dans rdflib, la serialisation sera plus compacte :
```turtle
<< ex:Alice foaf:knows ex:Bob >> ex:confidence "0.9"^^xsd:double .
```
au lieu des 6 triplets actuels (1 fait + 4 reification + 1 annotation).

---

## 8. Comparaison des trois approches

Recapitulons les trois mecanismes pour annoter des triplets RDF :

| Critere | Reification classique | Graphes nommes | RDF-Star / RDF 1.2 |
|---------|----------------------|----------------|--------------------|
| **Standard** | RDF 1.0 (2004) | SPARQL 1.1 (2013) | RDF 1.2 (2024) |
| **Granularite** | Par triplet | Par groupe | Par triplet |
| **Cout en triplets** | 4 par annotation | 1 URI par contexte | 0 (inline) |
| **Complexite SPARQL** | Jointures multiples | `GRAPH` clause | Pattern `<< >>` |
| **Support rdflib 7.x** | Complet | Complet (`Dataset`) | Non disponible |
| **Support triple stores** | Universel | Universel | Croissant |
| **Lisibilite** | Faible | Moyenne | Bonne |

### Recommandations

| Situation | Approche recommandee |
|-----------|---------------------|
| Annotation fine par triplet avec rdflib | Reification + fonction `reify()` |
| Provenance par lot ("ces 100 faits viennent de cette source") | Graphes nommes |
| Triple store supportant RDF-Star (Jena, Stardog, Oxigraph) | RDF-Star natif |
| Interoperabilite maximale | Reification (universellement supportee) |

---

## 9. Exercices

### Exercice 1 : Annoter des critiques de film avec la reification

Creez un graphe RDF representant les avis de trois critiques sur un film :
- Critique A : note 4/5, source "Le Monde", date 2024-01-15
- Critique B : note 3/5, source "Telerama", date 2024-01-20
- Critique C : note 5/5, source "Cahiers du Cinema", date 2024-02-01

Le fait principal est `ex:Film1 ex:rating ?note` avec les annotations de provenance.

**Indice** : Chaque critique produit un `rdf:Statement` different car la note differe.

En RDF-Star, cela s'ecrirait :
```turtle
<< ex:Film1 ex:rating "4"^^xsd:integer >> ex:reviewer "Critique A" ;
                                           ex:source "Le Monde" ;
                                           ex:date "2024-01-15"^^xsd:date .
```

In [15]:
# Exercice 1 : Critiques de film avec reification
from rdflib import Graph, Namespace, Literal, BNode
from rdflib.namespace import RDF, XSD

EX = Namespace("http://example.org/")

g_ex1 = Graph()
g_ex1.bind("ex", EX)

# Film
g_ex1.add((EX.Film1, RDF.type, EX.Film))
g_ex1.add((EX.Film1, EX.title, Literal("Les Temps Modernes")))

# TODO : Ajoutez les 3 critiques avec la fonction reify()
# Critique A : note 4/5
# note_a = Literal(4, datatype=XSD.integer)
# g_ex1.add((EX.Film1, EX.rating, note_a))
# stmt_a = reify(g_ex1, EX.Film1, EX.rating, note_a)
# g_ex1.add((stmt_a, EX.reviewer, Literal("Critique A")))
# g_ex1.add((stmt_a, EX.source, Literal("Le Monde")))
# g_ex1.add((stmt_a, EX.date, Literal("2024-01-15", datatype=XSD.date)))

# Critique B : note 3/5
# ...

# Critique C : note 5/5
# ...

# TODO : Affichez le graphe
# print(g_ex1.serialize(format="turtle"))

# TODO : Requete SPARQL pour lister les critiques
# query = """
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX ex: <http://example.org/>
# SELECT ?note ?reviewer ?source ?date
# WHERE {
#     ?stmt a rdf:Statement ;
#           rdf:subject ex:Film1 ;
#           rdf:predicate ex:rating ;
#           rdf:object ?note ;
#           ex:reviewer ?reviewer ;
#           ex:source ?source ;
#           ex:date ?date .
# }
# ORDER BY ?date
# """
# for row in g_ex1.query(query):
#     print(f"{row.reviewer} ({row.source}) : {row.note}/5 le {row.date}")

print("Completez l'exercice et decommentez le code.")

Completez l'exercice et decommentez le code.


### Exercice 2 : Requeter les reifications avec SPARQL

En utilisant le graphe de connaissances annote de la section 4.1 (variable `g_kg`), ecrivez des requetes SPARQL pour :

1. Trouver toutes les relations **assertees par une tierce personne** (c'est-a-dire qui ont un `ex:assertedBy`)
2. Trouver les relations **etablies depuis plus de 3 ans** (avant 2022-01-01)

**Indice** : Utilisez le pattern `?stmt a rdf:Statement ; rdf:subject ?s ; rdf:predicate foaf:knows ; rdf:object ?o .`

In [16]:
# Exercice 2 : Requetes SPARQL sur les reifications

# Requete 2a : Relations assertees par une tierce personne
query_2a = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

# TODO : Trouvez les relations avec ex:assertedBy
SELECT ?person1 ?person2 ?asserter
WHERE {
    # ?stmt a rdf:Statement ;
    #       rdf:subject ?person1 ;
    #       rdf:predicate foaf:knows ;
    #       rdf:object ?person2 ;
    #       ex:assertedBy ?asserter .
}
"""

# Requete 2b : Relations anciennes (avant 2022)
query_2b = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

# TODO : Trouvez les relations anterieures a 2022
SELECT ?person1 ?person2 ?since
WHERE {
    # ?stmt a rdf:Statement ;
    #       rdf:subject ?person1 ;
    #       rdf:predicate foaf:knows ;
    #       rdf:object ?person2 ;
    #       ex:since ?since .
    # FILTER(?since < "2022-01-01"^^xsd:date)
}
"""

# TODO : Decommentez et executez les requetes sur g_kg
# print("=== 2a : Relations assertees par un tiers ===")
# for row in g_kg.query(query_2a):
#     p1 = str(row.person1).split("/")[-1]
#     p2 = str(row.person2).split("/")[-1]
#     asserter = str(row.asserter).split("/")[-1]
#     print(f"  {asserter} affirme : {p1} connait {p2}")

# print("\n=== 2b : Relations anterieures a 2022 ===")
# for row in g_kg.query(query_2b):
#     p1 = str(row.person1).split("/")[-1]
#     p2 = str(row.person2).split("/")[-1]
#     print(f"  {p1} connait {p2} depuis {row.since}")

print("Completez les requetes et decommentez le code.")

Completez les requetes et decommentez le code.


---

## 10. Ecosysteme et compatibilite

RDF-Star est de plus en plus adopte par l'ecosysteme du Web Semantique.

### Support dans les triple stores

| Triple Store | Support RDF-Star | SPARQL-Star | Notes |
|-------------|-----------------|-------------|-------|
| **Apache Jena** (Fuseki) | Oui (depuis 4.2) | Oui | Implementation de reference |
| **Blazegraph** | Oui (extensions) | Oui | Support historique |
| **Stardog** | Oui (depuis 7.4) | Oui | Support commercial complet |
| **Oxigraph** | Oui (depuis 0.3) | Oui | Store Rust performant |
| **GraphDB** (Ontotext) | Oui (depuis 10.0) | Oui | Support entreprise |
| **Amazon Neptune** | Oui (depuis 2023) | Oui | Service cloud |
| **Virtuoso** | Partiel | Partiel | Via extensions |

### Support dans les bibliotheques

| Bibliotheque | Langage | RDF-Star natif | Reification | Version min |
|-------------|---------|---------------|-------------|-------------|
| **rdflib** | Python | Non (7.6.0) | Oui | Toutes |
| **Apache Jena** | Java | Oui | Oui | 4.2 |
| **dotNetRDF** | C# | Partiel | Oui | 3.x |
| **N3.js** | JavaScript | Oui | Oui | 1.16 |
| **RDF4J** | Java | Oui | Oui | 4.0 |

### Standardisation W3C

| Document | Statut (2024) |
|----------|---------------|
| RDF 1.2 Concepts | Candidate Recommendation |
| RDF 1.2 Turtle | Candidate Recommendation |
| RDF 1.2 N-Triples | Candidate Recommendation |
| SPARQL 1.2 Query | Working Draft |
| RDF 1.2 JSON-LD | En discussion |

---

## Resume

### Concepts cles

| Concept | Description |
|---------|-------------|
| **Reification classique** | Mecanisme RDF 1.0 avec `rdf:Statement` (4 triplets par assertion, verbeux mais universel) |
| **Graphes nommes** | Regrouper des triplets dans un contexte identifie par une URI (granularite par lot) |
| **Quoted Triple (RDF-Star)** | Triplet utilise comme sujet/objet : `<< s p o >>` (RDF 1.2, pas encore dans rdflib) |
| **Provenance** | Annoter qui a dit quoi, quand, avec quelle source |
| **Confiance** | Score de fiabilite attache a une assertion |
| **Annotations temporelles** | Validite dans le temps d'un fait |
| **Fusion multi-sources** | Conserver et departager les valeurs de sources differentes |

### Competences acquises

1. Comprendre les limites de la reification classique et la motivation de RDF-Star
2. Creer des annotations de triplets avec la reification et la fonction `reify()`
3. Utiliser les graphes nommes (`Dataset`) comme alternative pour le contexte
4. Interroger les annotations avec SPARQL standard (patterns de reification)
5. Appliquer ces patterns a des cas concrets (provenance, temporalite, multi-sources)
6. Comparer reification, graphes nommes et RDF-Star pour choisir l'approche adaptee

### Pour aller plus loin

- [W3C RDF 1.2 Concepts](https://www.w3.org/TR/rdf12-concepts/) - Specification officielle
- [W3C RDF 1.2 Turtle](https://www.w3.org/TR/rdf12-turtle/) - Syntaxe Turtle-Star
- [SPARQL 1.2 Query](https://www.w3.org/TR/sparql12-query/) - Extension SPARQL-Star
- [Olaf Hartig - RDF* and SPARQL*](https://arxiv.org/abs/1406.3399) - Article fondateur
- [rdflib documentation](https://rdflib.readthedocs.io/) - Guide rdflib

---

Le notebook suivant explore les graphes de connaissances (Knowledge Graphs), en combinant les technologies RDF, SPARQL et les patterns vus dans cette serie.

---

**Navigation** : [<< 10-JSONLD](SW-10-JSONLD.ipynb) | [Index](README.md) | [12-KnowledgeGraphs >>](SW-12-KnowledgeGraphs.ipynb)